# LSTM from Scratch for TA

## import packages

In [1]:
import numpy as np
from numpy import *
from numpy.random import *
import matplotlib.pyplot as plt
import random
import math
from datetime import datetime
import pickle
import pandas as pd
%matplotlib inline

## Define Function of Activation Function

In [2]:
#predefined activation function and its derivative
def Sigmoid(x): 
    return 1. / (1 + np.exp(-x))

def dSigmoid(values): 
    return values*(1-values)

def Tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def dtanh(values): 
    return 1. - values**2

def rand_arr(a, b, *args): 
    seed(0)
    return rand(*args)*(b - a) + a

## Define LSTM Class

In [3]:
class LSTM:
    def __init__(self, X_dim, y_dim, neuron):
        
        self.X_dim = X_dim 
        self.y_dim = y_dim
        self.neuron = neuron
        
        #initiate weight
        self.weight_f = rand_arr(-0.1, 0.1, neuron, X_dim + neuron)
        self.weight_i = rand_arr(-0.1, 0.1, neuron, X_dim + neuron)
        self.weight_g = rand_arr(-0.1, 0.1, neuron, X_dim + neuron) 
        self.weight_o = rand_arr(-0.1, 0.1, neuron, X_dim + neuron)
        self.weight_out = rand_arr(-0.1, 0.1, y_dim, neuron)
        
        #initiate bias
        self.bias_f = rand_arr(-0.1, 0.1, neuron) 
        self.bias_i = rand_arr(-0.1, 0.1, neuron) 
        self.bias_g = rand_arr(-0.1, 0.1, neuron) 
        self.bias_o = rand_arr(-0.1, 0.1, neuron)
        self.bias_out = rand_arr(-0.1, 0.1, y_dim)
    
    def forward(self, inputs, outputs):
        X = inputs 
        y = outputs
        
        self.X = X
        self.y = y
        #make list to save the values of every unit 
        self.X_concat = []
        self.f = []
        self.i = []
        self.g = []
        self.o = []
        self.s = [zeros(self.neuron)] #first s input is 0
        self.h = [zeros(self.neuron)] #first h input is 0
        self.out = []
        self.error = []
        for i in range(len(inputs)):
            #forward for every unit
            X_concat = np.hstack((X[i],  self.h[i]))
            self.X_concat.append(X_concat)
            self.f.append(Sigmoid(self.weight_f@X_concat + self.bias_f))
            self.i.append(Sigmoid(self.weight_i@X_concat + self.bias_i))
            self.g.append(Tanh(self.weight_g@X_concat + self.bias_g))
            self.o.append(Sigmoid(self.weight_o@X_concat + self.bias_o))
            self.s.append(self.f[i]*self.s[i] + self.i[i]*self.g[i])
            self.h.append(Tanh(self.s[i+1])*self.o[i])
            self.out.append(self.weight_out@self.h[i+1] + self.bias_out)
            self.error.append(abs(self.out[-1] - y[i]))
        #show error
        #print(np.array(self.error).mean())
    
    def backward(self, inputs, outputs):
        self.diff_out = []
        self.diff_h = []
        self.diff_h_bottom = [zeros(self.neuron)]
        self.diff_s_up = [zeros(self.neuron)]
        self.diff_s = []
        self.diff_o = []
        self.diff_g = []
        self.diff_i = []
        self.diff_f = []
        for i in range(len(inputs)-1,-1,-1):
            #derivate of output
            if i == len(inputs)-1:
                self.diff_out  = [2*(self.out[i] - outputs[i])] + self.diff_out
                self.diff_h = [self.diff_out[0]*dSigmoid(self.weight_out@self.h[i+1] + self.bias_out)*
                               self.weight_out + self.diff_h_bottom[0]] + self.diff_h
            else:
                self.diff_h = [self.diff_h_bottom[0]] + self.diff_h
            
            
            #derivative of gate/neural network operation
            self.diff_s = [self.diff_h[0]*self.o[i]*dtanh(self.s[i+1]) + self.diff_s_up[0]] + self.diff_s
            self.diff_o = [self.diff_h[0]*Tanh(self.s[i+1])] + self.diff_o
            self.diff_g = [self.i[i]*self.diff_s[0]] + self.diff_g
            self.diff_i = [self.g[i]*self.diff_s[0]] + self.diff_i
            self.diff_f = [self.s[i]*self.diff_s[0]] + self.diff_f
            
            #derivative of concatenation of input dan previous output value
            self.dX_concat = (self.diff_o[0].T*(dSigmoid(self.o[i]).reshape(self.neuron,1))).T@self.weight_o
            self.dX_concat += (self.diff_g[0].T*(dSigmoid(self.g[i]).reshape(self.neuron,1))).T@self.weight_g
            self.dX_concat += (self.diff_i[0].T*(dSigmoid(self.i[i]).reshape(self.neuron,1))).T@self.weight_i
            self.dX_concat += (self.diff_f[0].T*(dSigmoid(self.f[i]).reshape(self.neuron,1))).T@self.weight_f
            
            #update value of long and short term memory
            self.diff_h_bottom = [self.dX_concat[-1][self.X_dim:]] + self.diff_h_bottom
            self.diff_s_up = [self.diff_s[0]*self.f[i]] + self.diff_s_up
            
    def update(self, alpha):
        
        #alpha 
        
        #update everyweight and bias
        self.weight_out -= alpha*self.diff_out[0].reshape(1,1)@self.h[1].T.reshape(1,self.neuron)
        self.bias_out -= alpha*self.diff_out[0]
        
        self.weight_f -= alpha*(self.diff_f[0]*dSigmoid(self.weight_f@self.X_concat[0] + self.bias_f)).T@self.X_concat[0].reshape(1,self.X_dim + self.neuron)
        self.bias_f -= alpha*(self.diff_f[0]*dSigmoid(self.weight_f@self.X_concat[0] + self.bias_f)).reshape(self.neuron,)
        
        self.weight_i -= alpha*(self.diff_i[0]*dSigmoid(self.weight_i@self.X_concat[0] + self.bias_i)).T@self.X_concat[0].reshape(1,self.X_dim + self.neuron)
        self.bias_i -= alpha*(self.diff_i[0]*dSigmoid(self.weight_i@self.X_concat[0] + self.bias_i)).reshape(self.neuron,)
        
        self.weight_g -= alpha*(self.diff_g[0]*dSigmoid(self.weight_g@self.X_concat[0] + self.bias_g)).T@self.X_concat[0].reshape(1,self.X_dim + self.neuron)
        self.bias_g -= alpha*(self.diff_g[0]*dSigmoid(self.weight_g@self.X_concat[0] + self.bias_g)).reshape(self.neuron,)
        
        self.weight_o -= alpha*(self.diff_o[0]*dSigmoid(self.weight_o@self.X_concat[0] + self.bias_o)).T.reshape(self.neuron,1)@self.X_concat[0].reshape(1,self.X_dim + self.neuron)
        self.bias_o -= alpha*(self.diff_o[0]*dSigmoid(self.weight_o@self.X_concat[0] + self.bias_o)).reshape(self.neuron,)
    
    def predict(self, inputs, outputs):
        X = inputs
        y = outputs
        self.X = X
        self.y = y
        #make list to save the values of every unit 
        self.X_concat = []
        self.f = []
        self.i = []
        self.g = []
        self.o = []
        self.s = [zeros(self.neuron)] #first s input is 0
        self.h = [zeros(self.neuron)] #first h input is 0
        self.out = []
        self.error = []
        for i in range(len(inputs)):
            #forward for every unit
            X_concat = np.hstack((X[i],  self.h[i]))
            self.X_concat.append(X_concat)
            self.f.append(Sigmoid(self.weight_f@X_concat + self.bias_f))
            self.i.append(Sigmoid(self.weight_i@X_concat + self.bias_i))
            self.g.append(Tanh(self.weight_g@X_concat + self.bias_g))
            self.o.append(Sigmoid(self.weight_o@X_concat + self.bias_o))
            self.s.append(self.f[i]*self.s[i] + self.i[i]*self.g[i])
            self.h.append(Tanh(self.s[i+1])*self.o[i])
            self.out.append(self.weight_out@self.h[i+1] + self.bias_out)
            self.error.append(abs(self.out[-1] - y[i]))
        
        return [self.out[-1][0],self.y[-1]]
    
    def show_progress(self):
        return abs(self.y[-1][0] - self.out[-1])

## Change the Indicators Here!

In [4]:
#hyperparameters
epochs  = 1000
nc      = 10 #units

#datasets
name_dataset = 'PSDN-long'

#column_dataset_obs = 'Close'

## Import Dataset

In [5]:
df = pd.read_csv('../Datasets/'+name_dataset+'.csv')
# df.drop('Volume', inplace=True, axis=1)
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,02/01/2017,134,134,134,134,134,0
1,03/01/2017,135,139,135,139,139,4500
2,04/01/2017,139,139,136,136,136,40500
3,05/01/2017,135,141,135,141,141,6900
4,06/01/2017,136,140,136,140,140,200
...,...,...,...,...,...,...,...
1257,24/12/2021,153,157,151,151,151,26400
1258,27/12/2021,151,174,143,174,174,809900
1259,28/12/2021,175,176,162,162,162,914400
1260,29/12/2021,162,164,153,155,155,157400


In [6]:
df.describe()
new_df = df.drop(["Date","Adj Close"], axis=1)

In [26]:
new_df

,Open,High,Low,Close,Volume
0,134,134,134,134,0
1,135,139,135,139,4500
2,139,139,136,136,40500
3,135,141,135,141,6900
4,136,140,136,140,200
...,...,...,...,...,...
1257,153,157,151,151,26400
1258,151,174,143,174,809900
1259,175,176,162,162,914400
1260,162,164,153,155,157400


In [27]:
#plt.plot(new_df)


## MinMax Scaler

In [28]:
from sklearn.preprocessing import MinMaxScaler

# Get the number of rows in the data
nrows = new_df.shape[0]
print(nrows)
# Convert the data to numpy values
np_data_unscaled = np.array(new_df)
print(np_data_unscaled)
np_data = np.reshape(np_data_unscaled, (nrows, -1))
print(np_data.shape)

# Transform the data by scaling each feature to a range between 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
np_data_scaled = scaler.fit_transform(np_data_unscaled)

np_data_scaled

1262
[[   134    134    134    134      0]
 [   135    139    135    139   4500]
 [   139    139    136    136  40500]
 ...
 [   175    176    162    162 914400]
 [   162    164    153    155 157400]
 [   154    164    153    153  60600]]
(1262, 5)


array([[0.0624025 , 0.04888268, 0.07130125, 0.05799373, 0.        ],
       [0.06396256, 0.05586592, 0.07308378, 0.06583072, 0.00023366],
       [0.07020281, 0.05586592, 0.07486631, 0.06112853, 0.00210296],
       ...,
       [0.12636505, 0.1075419 , 0.12121212, 0.10188088, 0.04748009],
       [0.10608424, 0.09078212, 0.10516934, 0.09090909, 0.00817297],
       [0.09360374, 0.09078212, 0.10516934, 0.08777429, 0.00314665]])

In [29]:

# scaler = MinMaxScaler(feature_range=(0,1))
# new_df = scaler.fit_transform(np.array(new_df).reshape(-1,1))
# new_df

In [30]:
xs = np_data_scaled

In [31]:
xs

array([[0.0624025 , 0.04888268, 0.07130125, 0.05799373, 0.        ],
       [0.06396256, 0.05586592, 0.07308378, 0.06583072, 0.00023366],
       [0.07020281, 0.05586592, 0.07486631, 0.06112853, 0.00210296],
       ...,
       [0.12636505, 0.1075419 , 0.12121212, 0.10188088, 0.04748009],
       [0.10608424, 0.09078212, 0.10516934, 0.09090909, 0.00817297],
       [0.09360374, 0.09078212, 0.10516934, 0.08777429, 0.00314665]])

In [32]:
#Standarized the dataset
N       = xs.shape[0]

#hypreparameter
Ts      = 5
x_dim   = 1
alpha   = 0.01 #learningrate

xt      = xs[0:N-x_dim,:]

for i in range(x_dim):
    xt  = hstack((xt, xs[i+1:N-x_dim+i+1]))   



In [33]:
xt.shape[0]

1261

## Splitting Training and Testing

In [36]:
training_size = int(len(new_df)*0.8)
test_size = len(new_df)-training_size

In [37]:
training_size, test_size

(1009, 253)

In [38]:
X_train = xt[:training_size, 0:x_dim]     
y_train = xt[:training_size, x_dim:x_dim+1]  
X_test = xt[-test_size:, 0:x_dim]     
y_test = xt[-test_size:, x_dim:x_dim+1]

## Build LSTM Model

In [39]:
me = LSTM(x_dim,1,nc)
for i in range(epochs):
    print('-------------',i+1,'-------------')
    for j in range(X_train.shape[0]-Ts):
        me.forward(X_train[j:j+Ts],y_train[j:j+Ts])
        me.backward(X_train[j:j+Ts],y_train[j:j+Ts])
        me.update(alpha)
    hehe = []
    for j in range(X_test.shape[0]-Ts):
        me.forward(X_test[j:j+Ts],y_test[j:j+Ts])
        me.backward(X_test[j:j+Ts],y_test[j:j+Ts])
        me.update(alpha)
        hehe.append(me.show_progress())    
    print(np.array(hehe).mean())

------------- 1 -------------
0.0286437936464263
------------- 2 -------------
0.028635658663993247
------------- 3 -------------
0.028627526383908395
------------- 4 -------------
0.028619396799162783
------------- 5 -------------
0.028611269911184456
------------- 6 -------------
0.02860314572139641
------------- 7 -------------
0.028595024231216418
------------- 8 -------------
0.028586905442057095
------------- 9 -------------
0.028578789355325766
------------- 10 -------------
0.028570675972424416
------------- 11 -------------
0.028562565294749633
------------- 12 -------------
0.028554457323692592
------------- 13 -------------
0.028546352060638956
------------- 14 -------------
0.028538249506968833
------------- 15 -------------
0.028530149664056686
------------- 16 -------------
0.028522052533271357
------------- 17 -------------
0.02851395811597591
------------- 18 -------------
0.02850586641352769
------------- 19 -------------
0.028497777427278152
------------- 20 ---------

0.02740033748310459
------------- 159 -------------
0.02739263857833003
------------- 160 -------------
0.02738494249991166
------------- 161 -------------
0.02737724924793439
------------- 162 -------------
0.027369558822471985
------------- 163 -------------
0.02736187122358699
------------- 164 -------------
0.027354186451330705
------------- 165 -------------
0.02734650450574324
------------- 166 -------------
0.027338825386853373
------------- 167 -------------
0.027331149094678642
------------- 168 -------------
0.027323475629225255
------------- 169 -------------
0.027315804990488125
------------- 170 -------------
0.027308137178450816
------------- 171 -------------
0.027300472193085474
------------- 172 -------------
0.027292810034352927
------------- 173 -------------
0.02728515070220257
------------- 174 -------------
0.0272774941965724
------------- 175 -------------
0.027269840517388937
------------- 176 -------------
0.027262189664567293
------------- 177 -------------
0.

0.026231213723621603
------------- 316 -------------
0.026224071331409856
------------- 317 -------------
0.026216931588403374
------------- 318 -------------
0.026209794492822975
------------- 319 -------------
0.02620266004287757
------------- 320 -------------
0.026195528236763962
------------- 321 -------------
0.02618839907266694
------------- 322 -------------
0.02618127254875932
------------- 323 -------------
0.026174148663201842
------------- 324 -------------
0.026167027414143236
------------- 325 -------------
0.026159908799720324
------------- 326 -------------
0.02615279281805783
------------- 327 -------------
0.02614567946726857
------------- 328 -------------
0.026138568745453263
------------- 329 -------------
0.026131460650700787
------------- 330 -------------
0.026124355181087938
------------- 331 -------------
0.02611725233467965
------------- 332 -------------
0.026110152109528825
------------- 333 -------------
0.026103054503676407
------------- 334 -------------

0.025147645453121956
------------- 472 -------------
0.025140887564106058
------------- 473 -------------
0.02513413190711091
------------- 474 -------------
0.025127378478608894
------------- 475 -------------
0.025120627275062327
------------- 476 -------------
0.025113878292923563
------------- 477 -------------
0.02510713152863505
------------- 478 -------------
0.025100386978629128
------------- 479 -------------
0.025093644639328314
------------- 480 -------------
0.025086904507145173
------------- 481 -------------
0.02508016657848232
------------- 482 -------------
0.025073430849732552
------------- 483 -------------
0.025066697317278726
------------- 484 -------------
0.025059965977493895
------------- 485 -------------
0.025053236826741314
------------- 486 -------------
0.025046509861374328
------------- 487 -------------
0.025039785077736618
------------- 488 -------------
0.025033062472162013
------------- 489 -------------
0.025026342040974602
------------- 490 ----------

0.024120669949640416
------------- 628 -------------
0.024114272268379526
------------- 629 -------------
0.024107876145908785
------------- 630 -------------
0.024101481577435663
------------- 631 -------------
0.024095088558161188
------------- 632 -------------
0.024088697083279625
------------- 633 -------------
0.024082307147978915
------------- 634 -------------
0.024075918747440417
------------- 635 -------------
0.024069531876838995
------------- 636 -------------
0.02406314653134304
------------- 637 -------------
0.02405676270611456
------------- 638 -------------
0.02405038039630911
------------- 639 -------------
0.024043999597075878
------------- 640 -------------
0.024037620303557626
------------- 641 -------------
0.02403124251089092
------------- 642 -------------
0.02402486621420587
------------- 643 -------------
0.024018491408626358
------------- 644 -------------
0.024012118089269934
------------- 645 -------------
0.024005746251247975
------------- 646 ------------

0.023143240748483028
------------- 784 -------------
0.023137100500717458
------------- 785 -------------
0.02313096099037389
------------- 786 -------------
0.023124822211987247
------------- 787 -------------
0.0231186841600896
------------- 788 -------------
0.023112546829210243
------------- 789 -------------
0.023106410213875616
------------- 790 -------------
0.023100274308609423
------------- 791 -------------
0.023094139107932662
------------- 792 -------------
0.0230880046063635
------------- 793 -------------
0.02308187079841757
------------- 794 -------------
0.023075737678607558
------------- 795 -------------
0.023069605241443844
------------- 796 -------------
0.023063473481433942
------------- 797 -------------
0.023057342393082894
------------- 798 -------------
0.023051211970892934
------------- 799 -------------
0.023045082209364087
------------- 800 -------------
0.023038953102993545
------------- 801 -------------
0.02303282464627613
------------- 802 -------------


0.02219297454247249
------------- 940 -------------
0.02218691889336467
------------- 941 -------------
0.022180863110147362
------------- 942 -------------
0.022174807187231024
------------- 943 -------------
0.022168751119026962
------------- 944 -------------
0.02216269489994729
------------- 945 -------------
0.02215663852440502
------------- 946 -------------
0.022150590764357818
------------- 947 -------------
0.02214454818612151
------------- 948 -------------
0.022138505435173112
------------- 949 -------------
0.022132462505942734
------------- 950 -------------
0.02212641939286133
------------- 951 -------------
0.02212037609036096
------------- 952 -------------
0.022114332592874732
------------- 953 -------------
0.0221082888948367
------------- 954 -------------
0.022102244990682193
------------- 955 -------------
0.022096200874847517
------------- 956 -------------
0.02209015654177014
------------- 957 -------------
0.022084111985888683
------------- 958 -------------
0.0

In [48]:
X_train.shape[0]

1009

In [49]:
out

[[0.08635577519713183, array([0.04329609])],
 [0.08879269227233433, array([0.04329609])],
 [0.08997906115845584, array([0.04329609])],
 [0.09055529535417306, array([0.04329609])],
 [0.08892073856532916, array([0.03910615])],
 [0.08796261930176992, array([0.04329609])],
 [0.08954360766603421, array([0.04329609])],
 [0.08859142052115837, array([0.03212291])],
 [0.08772545525127236, array([0.02932961])],
 [0.08629708721526953, array([0.02793296])],
 [0.08583338961021171, array([0.03212291])],
 [0.085976533382192, array([0.02653631])],
 [0.08629818248367406, array([0.02793296])],
 [0.08603874256966038, array([0.02513966])],
 [0.08658182343882864, array([0.03910615])],
 [0.0849209907381939, array([0.03631285])],
 [0.0849757069941563, array([0.0349162])],
 [0.08477419862102334, array([0.0349162])],
 [0.08447807487174358, array([0.01955307])],
 [0.08347070172299018, array([0.02932961])],
 [0.08261550106821167, array([0.02653631])],
 [0.08239603024094105, array([0.02374302])],
 [0.081229261728

In [47]:
import matplotlib.pyplot as plt

out = []
for j in range(X_test.shape[0]-Ts):
    out.append(me.predict(X_test[j:j+Ts],y_test[j:j+Ts]))

out=scaler.inverse_transform(out.reshape())

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: operands could not be broadcast together with shapes (248,2) (5,) (248,2) 

## Report Performance of LSTM Predictions

In [ ]:
from math import sqrt

import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
# mse = mean_squared_error(y, predictions)
# print('MSE: '+str(mse))
print('epoch: ' + str(epochs))
print('units: ' + str(nc))

rmse = math.sqrt(mean_squared_error(np.array(out)[:,1], np.array(out)[:,0]))
print('RMSE: '+ str("{:.2f}".format(rmse)))
mae = mean_absolute_error(np.array(out)[:,1], np.array(out)[:,0])
print('MAE: '+ str("{:.2f}".format(mae)))
mape = mean_absolute_percentage_error(np.array(out)[:,1], np.array(out)[:,0])
print('MAPE: '+ str("{:.2f}".format(mape*100)))

## Visualize Test Data the Prediction and Real Stock

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [ ]:
plt.figure()
plt.plot(np.array(out)[:,1], color = 'red', label = 'Real Stock Price')
plt.plot(np.array(out)[:,0], color = 'blue', label = 'Predicted Stock Price')
plt.title(name_dataset +' Term Stock Prediction, epochs: '+ str(epochs) +'; units: '+str(nc))
plt.xlabel('Time')
plt.ylabel(name_dataset +' Stock Price '+ column_dataset_obs)
plt.legend()

## Save as a new Dataset

In [17]:
real_price = np.array(out)[:,1]
close_lstm = np.array(out)[:,0]

NameError: name 'out' is not defined

In [18]:
new_data = {'real_price': real_price,
            'Close_LSTM': close_lstm}


NameError: name 'real_price' is not defined

In [19]:
df_new_data = pd.DataFrame(new_data, columns = ['real_price', 'Close_LSTM'])


NameError: name 'new_data' is not defined

In [20]:
df_new_data.to_csv('../Datasets/dataset after prediction/'+name_dataset
                   +'_LSTM_'+ str(epochs) +'_'+ str(nc) + '.csv', index=False)

NameError: name 'df_new_data' is not defined